In [1]:
import requests
import time
import csv
import json
import os

In [ ]:
#configurar API key de Elsevier

API_KEY = "your_api_key_here"
BASE_URL = "https://api.elsevier.com/content"

HEADERS = {
    "X-ELS-APIKey": API_KEY,
    "Accept": "application/json"
}

In [3]:
# Función para buscar por palabras clave en Scopus

def scopus_search(query, start=0, count=25):
    url = f"{BASE_URL}/search/scopus"
    params = {
        "query": query,
        "start": start,
        "count": count,
        "sort": "-coverDate"
    }

    response = requests.get(url, headers=HEADERS, params=params)
    response.raise_for_status()
    print("Status:", response.status_code)
    print("Rate limit:", response.headers.get("X-RateLimit-Limit"))
    print("Remaining:", response.headers.get("X-RateLimit-Remaining"))
    print("Reset:", response.headers.get("X-RateLimit-Reset"))
    return response.json()


In [ ]:
#recuperar abstracts, autores, keywords y referencias
def get_abstract(scopus_id, sleep_time=0.3):
    url = f"{BASE_URL}/abstract/scopus_id/{scopus_id}"
    params = {
        "view": "FULL"
    }

    response = requests.get(url, headers=HEADERS, params=params)
    time.sleep(sleep_time)  # evita rate limit
    response.raise_for_status()
    return response.json()

#Extraer información relevante del JSON

def parse_abstract(json_data):
    core = json_data["abstracts-retrieval-response"]["coredata"]

    title = core.get("dc:title")
    abstract = core.get("dc:description")
    doi = core.get("prism:doi")
    year = core.get("prism:coverDate", "")[:4]

    # Autores
    authors = []
    author_group = json_data["abstracts-retrieval-response"].get("authors", {})
    for author in author_group.get("author", []):
        authors.append({
            "name": f"{author.get('given-name', '')} {author.get('surname', '')}",
            "scopus_id": author.get("@auid")
        })

    # Keywords
    keywords = set()

    auth_kw = json_data["abstracts-retrieval-response"].get("authkeywords", {})
    for kw in auth_kw.get("author-keyword", []):
        keywords.add(kw.get("$"))

    idx_kw = json_data["abstracts-retrieval-response"].get("idxterms", {})
    for kw in idx_kw.get("mainterm", []):
        keywords.add(kw.get("$"))

    # Referencias
    references = []
    ref_list = json_data["abstracts-retrieval-response"].get("references", {})
    for ref in ref_list.get("reference", []):
        references.append({
            "title": ref.get("ref-title", {}).get("ref-titletext"),
            "doi": ref.get("ref-info", {}).get("refdoi"),
            "year": ref.get("ref-publicationyear", {}).get("@first")
        })

    return {
        "title": title,
        "abstract": abstract,
        "doi": doi,
        "year": year,
        #"authors": authors,
        #"keywords": list(keywords),
        #"references": references
    }

In [5]:
def save_to_csv(records, filename="scopus_results.csv"):
    file_exists = os.path.isfile(filename)

    fieldnames = [
        "scopus_id",
        "doi",
        "year",
        "title",
        "abstract",
        "authors",
        "keywords",
        "references"
    ]

    with open(filename, mode="a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)

        if not file_exists:
            writer.writeheader()

        for r in records:
            writer.writerow({
                "scopus_id": r.get("scopus_id"),
                "doi": r.get("doi"),
                "year": r.get("year"),
                "title": r.get("title"),
                "abstract": r.get("abstract"),
                "authors": " | ".join(
                    a["name"] for a in r.get("authors", [])
                ),
                "keywords": "; ".join(r.get("keywords", [])),
                "references": json.dumps(
                    r.get("references", []),
                    ensure_ascii=False
                )
            })


In [ ]:
busquedas = ["active aging elderly",
             "active aging",
             "elderly", 
             "recommender systems human learning",  
             "ontology movement", 
             "human movement language models",  
             "tai chi",
             "tai chi ontology", 
             "pathology ontology",
             "tai chi disabilities"
             ]

n_results = 25 #es el max permitido por la API, para aprovechar bien las consultas permitidas
years_back = 5
max_results = 500
current_year = time.localtime().tm_year
ending_year = current_year - years_back

filenames = []

for termino_busqueda in busquedas:
    print(f"\n🔍 Buscando: {termino_busqueda}")
    starting_point = 0
    query = f'TITLE-ABS-KEY({termino_busqueda}) AND DOCTYPE(ar) AND PUBYEAR > {ending_year}'
    scopus_ids = []
    all_records = []
    while True:
        data = scopus_search(query, start=starting_point, count=n_results)
        total_available = int(data["search-results"]["opensearch:totalResults"])
        print(f"Total artículos disponibles en Scopus: {total_available}")

        entries = data["search-results"].get("entry", [])
        if not entries:
            break
        elif total_available <= len(scopus_ids):
            print('llegamos al final')
            break

        # extraer DOI y Scopus ID
        for entry in entries:
            scopus_id = entry["dc:identifier"].replace("SCOPUS_ID:", "")
            doi = entry.get("prism:doi")
            record = {
                "scopus_id": entry["dc:identifier"].replace("SCOPUS_ID:", ""),
                "doi": entry.get("prism:doi", ""),
                "title": entry.get("dc:title", ""),
                "year": entry.get("prism:coverDate", "")[:4],  # solo año
                "authors": "; ".join(a.get("authname") for a in entry.get("authors", [])) if "authors" in entry else "",
                "keywords": "; ".join(entry.get("authkeywords", [])) if "authkeywords" in entry else "",
                "references": entry.get("reference", "")
            }
            all_records.append(record)
            if doi:  # solo guardar si hay DOI
                scopus_ids.append((scopus_id, doi))

        # imprimir nuevos IDs
        print(f"IDs acumulados con DOI: {len(scopus_ids)}")

        if len(scopus_ids) >= max_results:
            print('llegamos al final')
            break

        starting_point += n_results

    # eliminar duplicados y limitar al máximo
    #scopus_ids = list(dict.fromkeys(scopus_ids))[:max_results]

    # guardar CSV
    filename = f"scopus_metadata_{termino_busqueda.replace(' ', '_')}.csv"
    filenames.append(filename)
    with open(filename, "w", newline="", encoding="utf-8") as f:
        fieldnames = ["scopus_id", "doi", "title", "year", "authors", "keywords", "references"]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for rec in all_records:
            writer.writerow(rec)

    print(f"💾 Guardados {len(scopus_ids)} DOIs en {filename}")
    


🔍 Buscando: recommender systems human learning


Status: 200
Rate limit: 20000
Remaining: 19866
Reset: 1768729911
Total artículos disponibles en Scopus: 464
IDs acumulados con DOI: 25
Status: 200
Rate limit: 20000
Remaining: 19865
Reset: 1768729911
Total artículos disponibles en Scopus: 464
IDs acumulados con DOI: 50
Status: 200
Rate limit: 20000
Remaining: 19864
Reset: 1768729911
Total artículos disponibles en Scopus: 464
IDs acumulados con DOI: 75
Status: 200
Rate limit: 20000
Remaining: 19863
Reset: 1768729911
Total artículos disponibles en Scopus: 464
IDs acumulados con DOI: 100
Status: 200
Rate limit: 20000
Remaining: 19862
Reset: 1768729911
Total artículos disponibles en Scopus: 464
IDs acumulados con DOI: 125
Status: 200
Rate limit: 20000
Remaining: 19861
Reset: 1768729911
Total artículos disponibles en Scopus: 464
IDs acumulados con DOI: 150
Status: 200
Rate limit: 20000
Remaining: 19860
Reset: 1768729911
Total artículos disponibles en Scopus: 464
IDs acumulados con DOI: 175
Status: 200
Rate limit: 20000
Remaining: 19859
Rese

In [8]:
filenames = []
filenames.append(filename)

In [ ]:
# hace falta más permisos para obtener los abstracts completos
"""for i, j in zip(filenames, busquedas):
    print(i, j)
    termino_busqueda = j
    input_file = i

    all_records = []

    
    with open(input_file, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)

        for idx, row in enumerate(reader):
            if idx >= max_results:
                break
            scopus_id = row["scopus_id"]

            try:
                raw = get_abstract(scopus_id)
                parsed = parse_abstract(raw)
                all_records.append(parsed)
                time.sleep(0.5)
            except Exception as e:
                print(f"Error con {scopus_id}: {e}")
    save_to_csv(all_records, "scopus_{}.csv".format(termino_busqueda.replace(" ", "_")))"""

'for i, j in zip(filenames, busquedas):\n    print(i, j)\n    termino_busqueda = j\n    input_file = i\n\n    all_records = []\n\n\n    with open(input_file, newline="", encoding="utf-8") as f:\n        reader = csv.DictReader(f)\n\n        for idx, row in enumerate(reader):\n            if idx >= max_results:\n                break\n            scopus_id = row["scopus_id"]\n\n            try:\n                raw = get_abstract(scopus_id)\n                parsed = parse_abstract(raw)\n                all_records.append(parsed)\n                time.sleep(0.5)\n            except Exception as e:\n                print(f"Error con {scopus_id}: {e}")\n    save_to_csv(all_records, "scopus_{}.csv".format(termino_busqueda.replace(" ", "_")))'